In [2]:
# === P-HSUMO — Percentiles de la MÁXIMA suma 30d por cliente (Outbound Cash) ==
# LÓGICA EXACTA (parametrización):
#   tx_direction = Outbound AND tx_type = Cash
#   Por cliente: S30(t) = suma base en los últimos 30 días (incluye t)
#   Tomamos max_{t}(S30) por cliente y calculamos percentiles sobre esos máximos.

import pandas as pd, numpy as np

pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

# -------- EDITA AQUÍ -----------------------------------------------------------
PATH = "../../data/tx/transacciones_cash_2025__with_subsub.csv"
SUBSUBSEGMENTS = "I-2"   # <-- ajusta el sub-subsegmento
PCTS = [0.85, 0.90, 0.95, 0.97, 0.99]
# ------------------------------------------------------------------------------

# Carga y limpieza mínima
df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_date_time"]   = pd.to_datetime(df["tx_date_time"], errors="coerce")
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")
df["tx_direction"]   = df.get("tx_direction","").astype(str).str.title()
df["tx_type"]        = df.get("tx_type","").astype(str).str.title()

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS)) 

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

# Filtro: OUT + Cash + datos válidos
g = df[
    df["tx_direction"].eq("Outbound")
    & df["tx_type"].eq("Cash")
    & df["tx_date_time"].notna()
    & df["tx_base_amount"].notna()
][["customer_id","tx_date_time","tx_base_amount"]].copy()

if g.empty:
    print("P-HSUMO: No hay transacciones elegibles.")
else:
    max_rows = []
    for cid, sub in g.groupby("customer_id", sort=False):
        daily = (sub.set_index("tx_date_time")["tx_base_amount"]
                   .abs()
                   .resample("D").sum())
        if daily.empty:
            continue
        S30 = daily.rolling("30D").sum()
        max_rows.append({"customer_id": cid, "S30_max": float(S30.max())})

    R = pd.DataFrame(max_rows)
    if R.empty:
        print("P-HSUMO: No se pudieron construir ventanas 30d.")
    else:
        s = R["S30_max"].astype(float)
        q = s.quantile(PCTS) if len(s) else pd.Series(index=PCTS, dtype=float)

        out = pd.DataFrame({
            "percentil": [f"p{int(p*100)}" for p in PCTS],
            "Amount_30d_max_per_customer_CLP": [q.get(p, np.nan) for p in PCTS]
        })

        print("=== P-HSUMO — Máxima suma CLP en 30 días por cliente (percentiles) ===")
        print(f"Clientes considerados: {s.shape[0]:,}")
        display(out)

        if pd.notna(q.get(0.95, np.nan)):
            print(f"\nSugerencia {{var.Amount}} (p95 máx-30d por cliente): {q.get(0.95):,.0f} CLP")


C:\Users\vinso\AppData\Local\Temp\ipykernel_19868\692485672.py:18: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")


=== P-HSUMO — Máxima suma CLP en 30 días por cliente (percentiles) ===
Clientes considerados: 18


,percentil,Amount_30d_max_per_customer_CLP
0,p85,"14,550,475,150"
1,p90,"15,664,202,920"
2,p95,"16,938,751,151"
3,p97,"17,758,890,427"
4,p99,"18,579,029,703"



Sugerencia {var.Amount} (p95 máx-30d por cliente): 16,938,751,151 CLP


# Simulación alertas

In [12]:
# === P-HSUMO — Simulación (ventanas cliente–día) ===============================
# Regla: OUT + Cash; S30 > Amount
# Unidad = ventanas cliente–día. Opcional: colapsar rachas para 1 alerta por cliente.

import pandas as pd, numpy as np
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

# -------- EDITA AQUÍ -----------------------------------------------------------
PATH = "../../data/tx_iv.csv"     # MISMO CSV que parametrización
PARAMS = {
    "Actual": {"Amount": 9_087_500_000},  # igual que p90
    "p95": {"Amount": 838_500_000},
    "p97": {"Amount": 1_422_450_000},
    "p99": {"Amount": 3_327_800_000},
}
COLAPSAR_RACHAS = False  # True => cuenta 1er día de cada racha por cliente
# ------------------------------------------------------------------------------

df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_date_time"]   = pd.to_datetime(df["tx_date_time"], errors="coerce")
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")
df["tx_direction"]   = df["tx_direction"].astype(str).str.title()
df["tx_type"]        = df["tx_type"].astype(str).str.title()

g = df[
    df["tx_direction"].eq("Outbound") &
    df["tx_type"].eq("Cash") &
    df["tx_date_time"].notna() &
    df["tx_base_amount"].notna()
][["customer_id","tx_date_time","tx_base_amount"]].copy()

parts=[]
for cid, sub in g.groupby("customer_id", sort=False):
    daily = (sub.set_index("tx_date_time")["tx_base_amount"]
                .abs()
                .resample("D").sum())
    if daily.empty: 
        continue
    S30 = daily.rolling("30D").sum()
    parts.append(pd.DataFrame({"customer_id": cid, "date": S30.index, "S30": S30.values}))

M = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=["customer_id","date","S30"])

def contar_alertas(dfm, amount, colapsar_rachas=False):
    m = dfm["S30"] > amount
    if not colapsar_rachas:
        return int(dfm.loc[m, ["customer_id","date"]].drop_duplicates().shape[0])
    # Colapsar rachas contiguas por cliente (1ª fecha de cada racha)
    df2 = dfm.loc[m, ["customer_id","date"]].sort_values(["customer_id","date"])
    # Una racha empieza cuando no hay día anterior consecutivo para el mismo cliente
    df2["prev_date"] = df2.groupby("customer_id")["date"].shift(1)
    df2["is_new_run"] = (df2["prev_date"].isna()) | ((df2["date"] - df2["prev_date"]).dt.days > 1)
    return int(df2.loc[df2["is_new_run"], ["customer_id","date"]].shape[0])

order = ["Actual","p90","p95","p97","p99"]
param_tbl = pd.DataFrame(PARAMS).T.loc[[k for k in order if k in PARAMS]].rename_axis("escenario").reset_index()
print("=== P-HSUMO — Parámetros (Amount) ==="); display(param_tbl)

counts = {k: contar_alertas(M, v["Amount"], COLAPSAR_RACHAS) for k,v in PARAMS.items()}
out = pd.DataFrame([{
    "alertas_" + k: counts.get(k, 0) for k in order
}])
print(f"=== P-HSUMO — Alertas por escenario (ventanas; colapsar_rachas={COLAPSAR_RACHAS}) ===")
display(out)


=== P-HSUMO — Parámetros (Amount) ===


,escenario,Amount
0,Actual,9087500000
1,p95,838500000
2,p97,1422450000
3,p99,3327800000


=== P-HSUMO — Alertas por escenario (ventanas; colapsar_rachas=False) ===


,alertas_Actual,alertas_p90,alertas_p95,alertas_p97,alertas_p99
0,293,0,3750,2140,824


In [19]:
# === P-HSUMO — Simulación de alertas (Actual vs propuestos) ====================
# LÓGICA EXACTA (simulación):
#   tx_direction = Outbound AND tx_type = Cash
#   sum of tx_base_amount per {customer_id & tx_direction} in 30 days > [Amount]
# Unidad = ventanas (customer_id, día) que cumplen.

import pandas as pd, numpy as np
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

# ---- EDITA AQUÍ ----------------------------------------------------------------
PATH = "../../data/tx_retail_whale.csv"   # <-- usa el MISMO CSV que en la parametrización
PARAMS = {
    "Actual": {"Amount": 373_000_000},
    "p95":    {"Amount": 1_271_700_000},
    "p97":    {"Amount": 1_723_000_000},
    "p99":    {"Amount": 1_958_700_000},
}
# -----------------------------------------------------------------------------

# Carga + filtros mínimos
df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_date_time"]   = pd.to_datetime(df.get("tx_date_time"), errors="coerce")
df["tx_base_amount"] = pd.to_numeric(df.get("tx_base_amount"), errors="coerce")
df["tx_direction"]   = df.get("tx_direction","").astype(str).str.title()
df["tx_type"]        = df.get("tx_type","").astype(str).str.title()

g = df[
    df["tx_direction"].eq("Outbound")
    & df["tx_type"].eq("Cash")
    & df["tx_date_time"].notna()
    & df["tx_base_amount"].notna()
    & df["customer_id"].notna()
][["customer_id","tx_date_time","tx_base_amount"]].copy()

if g.empty:
    print("No hay transacciones elegibles para P-HSUMO.")
else:
    parts = []
    for cid, sub in g.groupby("customer_id", sort=False):
        daily = (sub.set_index("tx_date_time")["tx_base_amount"]
                   .abs()
                   .resample("D").sum())
        if daily.empty:
            continue
        S30 = daily.rolling("30D", min_periods=1).sum()
        parts.append(pd.DataFrame({
            "customer_id": cid,
            "date": S30.index,
            "S30":  S30.values
        }))

    M = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=["customer_id","date","S30"])

    order = ["Actual","p85","p90","p95","p97","p99"]
    param_tbl = (pd.DataFrame(PARAMS).T
                    .loc[[k for k in order if k in PARAMS]]
                    .rename_axis("escenario")
                    .reset_index())
    print("=== P-HSUMO — Parámetros (Amount) ==="); display(param_tbl)

    counts = {}
    for k, v in PARAMS.items():
        A = v["Amount"]
        m_ok = (M["S30"] > A)
        counts[k] = int(M.loc[m_ok, ["customer_id","date"]].drop_duplicates().shape[0])

    out = pd.DataFrame([{
        "alertas_actual": counts.get("Actual",0),
        "alertas_p85":    counts.get("p85",0),
        "alertas_p90":    counts.get("p90",0),
        "alertas_p95":    counts.get("p95",0),
        "alertas_p97":    counts.get("p97",0),
        "alertas_p99":    counts.get("p99",0),
    }])
    print("=== P-HSUMO — Alertas por escenario (ventanas cliente–día) ==="); display(out)


=== P-HSUMO — Parámetros (Amount) ===


,escenario,Amount
0,Actual,373000000
1,p95,1271700000
2,p97,1723000000
3,p99,1958700000


=== P-HSUMO — Alertas por escenario (ventanas cliente–día) ===


,alertas_actual,alertas_p85,alertas_p90,alertas_p95,alertas_p97,alertas_p99
0,505,0,0,62,31,1
